In [33]:
import pandas as pd
import numpy as np
import glob
from mysql import connector
# putting all csv files into one list
csv_files = [
    r"..\Data\ap_bus_details.csv",
    r"..\Data\Assam_bus_details.csv",
    r"..\Data\HimachalPradesh_bus_details.csv",
    r"..\Data\JammuKashmir_bus_details.csv",
    r"..\Data\Kadamba_bus_details.csv",
    r"..\Data\Kerala_bus_details.csv",
    r"..\Data\NorthBengal_bus_details.csv",
    r"..\Data\Punjab_bus_details.csv",
    r"..\Data\Rajasthan_bus_details.csv",
    r"..\Data\SouthBengal_bus_details.csv",
    r"..\Data\Telangana_bus_details.csv",
    r"..\Data\UttaraPradesh_bus_details.csv",
    r"..\Data\WestBengal_bus_details.csv"
]
df = [pd.read_csv(file) for file in csv_files]

# combine all dataframes in the list
combine_df = pd.concat(df, ignore_index=True)
combine_df.to_csv("Buses_data.csv", index=False)

df = pd.read_csv(r"..\Data\Buses_data.csv")
df = df.dropna() # drop null values

#add id column
id_column = pd.Series(range(1, len(df) + 1), name='id')
df = pd.concat([id_column, df], axis=1)
df = df.drop('Unnamed: 0',axis=1)
df['Price'] = df['Price'].str.replace('INR ', '', regex=False)
df['Price'] = pd.to_numeric(df['Price'])
df['Seat_Availability'] = df['Seat_Availability'].str.replace(r' Seat[s]? available', '', regex=True)
df['Seat_Availability'] = pd.to_numeric(df['Seat_Availability'])
connection = connector.connect(
    host = "localhost",
    user = "root",
    password = "root"
)
my_cursor=connection.cursor()
query = "use com_guvi_redbus"
my_cursor.execute(query)
table_name = "RedBus"
def to_sql(df):
    df = df.convert_dtypes()
    
    a = ",".join(f"{i} {j}"
                 for i,j in zip(df.columns,df.dtypes)).replace("string","varchar(200)").replace("float64","float").replace("object","text").replace("Int64","int").replace("int64","int").replace("Float64","float") #replace dtypes from pandas to mysql
    
    query = f"create table if not exists {table_name} ({a})"
    my_cursor.execute(query)
def insert_data():
    for i in df.values:
        my_cursor.execute(f"insert into {table_name} values {tuple(i)}")
        connection.commit()
to_sql(df)
insert_data()